In [1]:
import numpy 
import scipy
from scipy.signal import find_peaks
import itertools, sys, os, copy, pickle
sys.path.append('/mnt/home/szheng/projects/util_code')



import data_prep_new as dpn
import place_cell_analysis as pa
import plot_helper as ph
import plot_raster as pr
from importlib import reload
import matplotlib.pyplot as plt
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm
import jax
import jax.numpy as np
import jax.scipy as scipy
from jax import value_and_grad, grad, jit, vmap, jacfwd, jacrev
from jax.example_libraries import optimizers as jax_opt

import gm
import gm_plot as gp
import gm_plot_diagnostic as gpd
import seaborn as sns
%matplotlib inline

In [2]:
sess_name="e15_13f1_220117"
py_data_dir = "/mnt/home/szheng/ceph/ad/roman_data/e15/e15_13f1/e15_13f1_220117/py_data"
fr_map_ = pickle.load(open(os.path.join(py_data_dir,'fr_map.p'),'rb'))

# %%
fr_map=fr_map_['fr_map']
fr_map_trial=fr_map_['fr_map_trial']

args=pr.preprocess_for_plot(sess_name,sigma=30,speedmask=5,n_pos_bins=100)
section_colordict,spk_triggered_positions_trial_all_speedmasked,pos_bins_dict,choice = args[4:8]
bin_to_lin=copy.copy(pos_bins_dict['lin'])
bin_to_lin=np.concatenate([bin_to_lin[:-1,None],bin_to_lin[1:,None]],axis=1).mean(axis=1)

ind = 0
uid_l = fr_map[0].index
uid = uid_l[ind] # 1 indexed
uid_num = int(uid.split('_')[1])-1 # 0 indexed

ch=0
trial_mask = choice==ch

spk_triggered_positions_trial_one_unit = spk_triggered_positions_trial_all_speedmasked[uid_num]
spk_triggered_positions_trial  = spk_triggered_positions_trial_one_unit[trial_mask]


2022-06-28 21:32:36.830282: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


# CV

In [3]:
reload(gm)
ys_l = fr_map_trial[0][0]

In [20]:
import gm_cv as gc
reload(gc)

<module 'gm_cv' from '/mnt/home/szheng/projects/util_code/gm_cv.py'>

In [9]:
reg_pars_tosweep_kwargs_=dict(g_w=[10000],nfields=[2],g_mu=[0.1],g_sigma=[1,2])
cv_kwargs_=dict(cv_fold=3)
reg_pars_and_loss_df,pars_learned_l,mask_l,best_reg_par,best_pars_learned = gc.cv_one_neuron(ys_l,reg_pars_tosweep_kwargs_=reg_pars_tosweep_kwargs_,cv_kwargs_=cv_kwargs_,dosave=True)



0
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 1, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 2, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
1
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 1, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 2, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
2
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 1, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 0.1, 'g_sigma': 2, 'g_w': 10000, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
saved at cv_one_neuron.p


In [23]:
%%time
gc.cv_one_neuron_wrapper(1,0,testmode=1,forcereload=True)


0
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 1, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 2, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
1
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 1, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 2, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
2
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 1, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
{'S_l1': 100000000.0, 'g_mu': 200.0, 'g_sigma': 2, 'g_w': 10000000.0, 'g_b': 10000000.0, 'sigma_thresh': 1, 'g_sigma_thresh': 1000.0, 'nfields': 2}
saved at ch_1_ind_0.p
CPU times: user 19.5 s, sys: 34.7 ms, total: 19.5 s
Wall time: 19.6 s


In [13]:
os.path.exists('cv_one_neuron.p')

True

In [49]:
reg_pars_and_loss_df.groupby('reg_par_index')['test_loss'].mean().idxmin()

0

In [54]:
reg_pars_and_loss_df.query('reg_par_index==0').iloc[0][['g_w','g_sigma']]

g_w        10000.0
g_sigma        1.0
Name: 0, dtype: float64

In [45]:
pars_learned_l[4][1]

{'S': DeviceArray([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'logb': DeviceArray([ 0.50476056,  2.2736332 ,  1.0638167 ,  1.9805799 ,
               2.1026702 ,  1.6728783 ,  3.1514792 ,  2.55382   ,
               3.0820253 ,  2.1872807 ,  0.4000769 ,  0.07916217,
               0.32539636, -3.2796242 , -0.5201253 ,  0.5947417 ,
               3.6804173 ,  1.7764351 ,  1.3009468 ,  1.3831818 ,
               2.688259  ], dtype=float32),
 'logsigmas': DeviceArray([[48.999596 ,  5.306313 ],
              [49.000813 ,  5.273943 ],
              [48.998867 ,  4.924174 ],
              [48.98414  ,  4.5863867],
              [48.96532  ,  4.6305604],
              [48.94035  ,  4.55361  ],
              [48.911106 ,  4.445707 ],
              

In [26]:
!python ../../util_code/gm_cv.py

submitit executor will schedule jobs on slurm!
all jobs submitted
Job 1454786_0 === ch: 0, ind: 0
Job 1454786_1 === ch: 0, ind: 1
Job 1454786_2 === ch: 0, ind: 2
Job 1454786_3 === ch: 0, ind: 3
Job 1454786_4 === ch: 0, ind: 4
Job 1454786_5 === ch: 0, ind: 5
Job 1454786_6 === ch: 0, ind: 6
Job 1454786_7 === ch: 0, ind: 7
Job 1454786_8 === ch: 0, ind: 8
Job 1454786_9 === ch: 0, ind: 9
Job 1454786_10 === ch: 0, ind: 10
Job 1454786_11 === ch: 0, ind: 11
Job 1454786_12 === ch: 0, ind: 12
Job 1454786_13 === ch: 0, ind: 13
Job 1454786_14 === ch: 0, ind: 14
Job 1454786_15 === ch: 0, ind: 15
Job 1454786_16 === ch: 0, ind: 16
Job 1454786_17 === ch: 0, ind: 17
Job 1454786_18 === ch: 0, ind: 18
Job 1454786_19 === ch: 0, ind: 19
Job 1454786_20 === ch: 0, ind: 20
Job 1454786_21 === ch: 0, ind: 21
Job 1454786_22 === ch: 0, ind: 22
Job 1454786_23 === ch: 0, ind: 23
Job 1454786_24 === ch: 0, ind: 24
Job 1454786_25 === ch: 0, ind: 25
Job 1454786_26 === ch: 0, ind: 26
Job 1454786_27 === ch: 0, ind: 27
Jo